In [1]:
#import multiple files with names as Kickstarter*.csv into pandas dataframe (data used 15 Aug 2019)

from glob import glob
import pandas as pd

# for auto-reloading extenrnal modules
%load_ext autoreload
%autoreload 2

#just replace the path parts
#glob import all files starting with Kickstarter in the folder (the folder i used is the 15 Aug 2019)

filenames = glob(r"Kickstarter_2019-08-15T03_20_03_022Z/Kickstarter*.csv")

df = [pd.read_csv(f, encoding='utf-8') for f in filenames]

data = pd.concat(df, axis = 0, ignore_index = True).reset_index()

In [2]:
from pandas.io.json import json_normalize
import json as json
import numpy as np

#data['category'].fillna(json.dumps('{}'), inplace = True)
data['location'].fillna('{}', inplace = True)
#data['creator'].fillna(json.dumps('{}'), inplace = True)

x1 = json_normalize(data['category'].apply(json.loads)).add_prefix('category_')
x3 = json_normalize(data['location'].apply(json.loads)).add_prefix('location_')
x5 = json_normalize(data['creator'].apply(lambda x: json.loads(x.replace('\n', '').replace('\r', '').replace('\\', '').replace('"', r'\"').replace(r'{\"', '{"').replace(r'\"}', '"}').replace(r':\"', ':"').replace(r'\":', '":').replace(r',\"', ',"').replace(r'\",', '",').replace(r'", ', r'\",')))).add_prefix('creator_')

data = pd.concat([data, x1, x3, x5], axis = 1)

In [3]:
from datetime import datetime
data['launched_at_utc'] = data['launched_at'].apply(datetime.utcfromtimestamp)
data['deadline_utc'] = data['deadline'].apply(datetime.utcfromtimestamp)
data['created_at_utc'] = data['created_at'].apply(datetime.utcfromtimestamp)
data['goal'] = round(data['goal'] * data['static_usd_rate'], 2)

In [28]:
#pick the relevant data and drop all irrelevant data 

df = data[['backers_count', 
'goal', 'staff_pick', 'state', 'location_id',  'category_id', 'deadline_utc', 'launched_at_utc']]

#feature creation 
#feature 1: no. of projects for each creator 
# df1 = df.groupby('creator_id')['id'].count()
# df = pd.merge(df, pd.DataFrame(df1), how = 'left', on = 'creator_id')

#feature 2: duration of project 
df['duration'] = (df['deadline_utc'] - df['launched_at_utc']).dt.days

#feature 4: month of launch
df['launch_month'] = df['launched_at_utc'].dt.month

#change state to numerical 
df['state'] = df['state'].map({'successful': 1, 'failed': 0 })

#change staff_pick to numerical 
df['staff_pick'] = df['staff_pick'].map({True: 1, False: 0 })

#drop columns
df = df.drop(columns = ['launched_at_utc', 'deadline_utc'])
df = df.dropna() #drop null rows
columns = df.columns.tolist()

columns = columns[-4:] + columns[:-4]
df = df[columns]

/home/muruges/.conda/envs/CS3244_Proj/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/home/muruges/.conda/envs/CS3244_Proj/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/home/muruges/.conda/envs/CS3244_Proj/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

In [31]:
print(df.shape)
print(df)
print(np.array(df.iloc[:, 0:3].values))
print(df.columns.values.tolist())

X = np.array(df.iloc[:,:-1].values)
Y = np.array(df.iloc[:,-1].values)
print(X)
print(Y)

(192137, 8)
        location_id  category_id  duration  launch_month  backers_count  \
0         2483145.0           27        29             2             10   
1         2406080.0          291        30             2              2   
2         2357024.0           50        29             8             11   
4         2510436.0           27        30             6              3   
5         2430300.0           50        57             7             29   
...             ...          ...       ...           ...            ...   
207616   12485627.0           54        58             5             10   
207617    2367105.0          343        30            12             23   
207618      19344.0          249        20             7             61   
207619     673361.0          280        60             9              8   
207620     901163.0           26        14            10             22   

             goal  staff_pick  state  
0          500.00           0    0.0  
1       1